# Comparing different Neural Networks

Each Network uses a vector of the eigen-eigen parts and the eigenvalues and computes the rest-eigen contribution

In [ ]:
using LmaPredict, Flux, Statistics, Plots, Random, StatsBase, JLD2, PrettyTables, CSV, Tables, LaTeXStrings, Dates

## Reading the data

In [ ]:
const path_config = "/Users/lukasgeyer/Studium/Computational Sciences/Masterarbeit/Daten Simon/dat"
const path_plot = "../plots"

In [ ]:
fname = readdir(path_config)[2:5001]
idx = sortperm( parse.(Int64, fname))
fname = fname[idx]
em_n = "VV"

cnfgarr = Vector{LMAConfig}(undef, 0)
for f in fname
    push!(cnfgarr, get_LMAConfig(joinpath(path_config, f), "g5-g5", em=em_n, bc=false))
end

## Splitting data in training and test sets

In [ ]:
k_fold = 10
NCNFG = length(cnfgarr)
train_size = 500
test_size = NCNFG - train_size

TSRC = "24"
TVALS = length(cnfgarr[1].data["rr"][TSRC]) - 1
if em_n == "PA"
    EIGVALS = 32
else 
    EIGVALS = 64
end

eigvals_data_train = [Array{Float64}(undef, EIGVALS, train_size) for j in 1:k_fold]
rr_data_train = [Array{Float64}(undef, TVALS, train_size) for j in 1:k_fold]
ee_data_train = [Array{Float64}(undef, TVALS, train_size) for j in 1:k_fold]
re_data_train = [Array{Float64}(undef, TVALS, train_size) for j in 1:k_fold]

eigvals_data_test = [Array{Float64}(undef, EIGVALS, test_size) for j in 1:k_fold]
rr_data_test = [Array{Float64}(undef, TVALS, test_size) for j in 1:k_fold]
ee_data_test = [Array{Float64}(undef, TVALS, test_size) for j in 1:k_fold]
re_data_test = [Array{Float64}(undef, TVALS, test_size) for j in 1:k_fold]

for i in 1:k_fold
    all_indexes = [index for index in 1:NCNFG]
    train_start = ((i-1) * train_size) + 1
    train_end = (train_start + train_size) - 1

    for (k, dd) in enumerate(getfield.(cnfgarr, :data)[train_start:train_end])
        
        eigvals_data_train[i][:,k] = copy(cnfgarr[k].data["eigvals"][1:EIGVALS])
        
        rr_data_train[i][:,k] = getindex(getindex(dd, "rr"), TSRC)[2:end]

        re_data_train[i][:,k] = getindex(getindex(dd, "re"), TSRC)[2:end]
    
        ee_all_TSRC = Matrix{Float64}(undef, TVALS, TVALS)
        for ee_TSRC in 0:TVALS-1
            ee_all_TSRC[:,ee_TSRC+1] = getindex(getindex(dd, "ee"), "$ee_TSRC")[2:end]
        end

        ee_data_train[i][:,k] = mean(ee_all_TSRC, dims=2)
        #ee_data_train[i][:,k] = getindex(getindex(dd, "ee"), TSRC)[2:end]
    end
    
    for (k, dd) in enumerate(deleteat!(getfield.(cnfgarr, :data),[train_i for train_i in  train_start:train_end]))

        eigvals_data_test[i][:,k] = copy(cnfgarr[k].data["eigvals"][1:EIGVALS])

        rr_data_test[i][:,k] = getindex(getindex(dd, "rr"), TSRC)[2:end]
        
        re_data_test[i][:,k] = getindex(getindex(dd, "re"), TSRC)[2:end]
    
        ee_all_TSRC = Matrix{Float64}(undef, TVALS, TVALS)
        for ee_TSRC in 0:TVALS-1
            ee_all_TSRC[:,ee_TSRC+1] = getindex(getindex(dd, "ee"), "$ee_TSRC")[2:end]
        end

        ee_data_test[i][:,k] = mean(ee_all_TSRC, dims=2)
        #ee_data_test[i][:,k] = getindex(getindex(dd, "ee"), TSRC)[2:end]
    end
end

## Defining training and test data

In [ ]:
input_length = 2
output_length = 1

input_shape_train = [Array{Matrix{Float64}}(undef, TVALS) for k in 1:k_fold]
output_shape_train = [Array{Matrix{Float64}}(undef, TVALS) for k in 1:k_fold]
input_shape_test = [Array{Matrix{Float64}}(undef, TVALS) for k in 1:k_fold]

for k in 1:k_fold
    for i in 1:TVALS
        input_shape_train[k][i] = permutedims(hcat(ee_data_train[k][i,:], rr_data_train[k][i,:]))
        output_shape_train[k][i] = permutedims(reshape(re_data_train[k][i,:], :, 1))
        input_shape_test[k][i] = permutedims(hcat(ee_data_test[k][i,:], rr_data_test[k][i,:]))
    end
end

output_shape_test = [re_data_test[k] for k in 1:k_fold];

### Input

In [ ]:
input_data_train_standardized = [Array{Matrix{Float64}}(undef, TVALS) for k in 1:k_fold]
input_data_test_standardized = [Array{Matrix{Float64}}(undef, TVALS) for k in 1:k_fold]

fir k in 1:k_fold
    for i in 1:TVALS
        mean_input_train = mean(input_shape_train[k][i], dims=ndims(input_shape_train[k][i]))
        std_input_train = std(input_shape_train[k][i], dims=ndims(input_shape_train[k][i]))
            
        input_data_train_standardized[k][i] = (input_shape_train[k][i] .- mean_input_train) ./ std_input_train
        input_data_test_standardized[k][i] = (input_shape_test[k][i] .- mean_input_train) ./ std_input_train
    end
end

### Output

In [ ]:
output_data_train_standardized = [Array{Matrix{Float64}}(undef, TVALS) for k in 1:k_fold]

for k in 1:k_fold
    for i in 1:TVALS
        mean_output_train = mean(output_shape_train[k][i])
        std_output_train = std(output_shape_train[k][i])
    
        output_data_train_standardized[k][i] = (output_shape_train[k][i] .- mean_output_train) ./ std_output_train
    end
end

## Describing different Neural Networks

In [ ]:
activation_functions = [NNlib.tanh, NNlib.celu, NNlib.elu, NNlib.tanhshrink];

In [ ]:
models_perTimeSlice = []

for activation_function in activation_functions
    push!(models_perTimeSlice,
        [
    Chain(
    Dense(input_length => output_length, identity),
    ),
    Chain(
    Dense(input_length => 1, activation_function),
    Dense(1 => output_length, identity),
    ),
    Chain(
    Dense(input_length => 8, activation_function),
    Dense(8 => 8, activation_function),
    Dense(8 => 1, identity)
    ),
    Chain(
    Dense(input_length => 50, activation_function),
    Dense(50 => output_length, identity),
    ),
    Chain(
    Dense(input_length => 100, activation_function),
    Dense(100 => output_length, identity),
    ),
    Chain(
    Dense(input_length => 400, activation_function),
    Dropout(0.8),
    Dense(400 => output_length, identity)
    ),
    Chain(
    Dense(input_length => 600, activation_function),
    Dropout(0.8),
    Dense(600 => output_length, identity)
    ),
    Chain(
    Dense(input_length => 800, activation_function),
    Dropout(0.8),
    Dense(800 => output_length, identity)
    ),
    Chain(
    Dense(input_length => 1000, activation_function),
    Dropout(0.8),
    Dense(1000 => output_length, identity)
    ),
    Chain(
    Dense(input_length => 1200, activation_function),
    Dropout(0.8),
    Dense(1200 => output_length, identity)
    ),
    Chain(
    Dense(input_length => 50, activation_function),
    Dense(50 => 1, activation_function),
    Dense(1 => output_length, identity),
    ),
    Chain(
    Dense(input_length => 50, activation_function),
    Dense(50 => 50, activation_function),
    Dense(50 => output_length, identity),
    ),
    Chain(
    Dense(input_length => 100, activation_function),
    Dense(100 => 50, activation_function),
    Dense(50 => output_length, identity),
    ),
    Chain(
    Dense(input_length => 100, activation_function),
    Dense(100 => 100, activation_function),
    Dense(100 => output_length, identity),
    ),
    Chain(
    Dense(input_length => 400, activation_function),
    Dropout(0.8),
    Dense(400 => 200, activation_function),
    Dropout(0.8),
    Dense(200 => output_length, identity)
    ),
    Chain(
    Dense(input_length => 600, activation_function),
    Dropout(0.8),
    Dense(600 => 400, activation_function),
    Dropout(0.8),
    Dense(400 => output_length, identity)
    ),
    Chain(
    Dense(input_length => 800, activation_function),
    Dropout(0.8),
    Dense(800 => 600, activation_function),
    Dropout(0.8),
    Dense(600 => output_length, identity)
    ),
    Chain(
    Dense(input_length => 1000, activation_function),
    Dropout(0.8),
    Dense(1000 => 800, activation_function),
    Dropout(0.8),
    Dense(800 => output_length, identity)
    ),
    Chain(
    Dense(input_length => 50, activation_function),
    Dense(50 => 50, activation_function),
    Dense(50 => 50, activation_function),
    Dense(50 => output_length, identity),
    ),
    Chain(
    Dense(input_length => 100, activation_function),
    Dense(100 => 100, activation_function),
    Dense(100 => 100, activation_function),
    Dense(100 => output_length, identity),
    ),
    Chain(
    Dense(input_length => 400, activation_function),
    Dropout(0.8),
    Dense(400 => 400, activation_function),
    Dropout(0.8),
    Dense(400 => 400, activation_function),
    Dropout(0.8),
    Dense(400 => output_length, identity),
    ),
    Chain(
    Dense(input_length => 600, activation_function),
    Dropout(0.8),
    Dense(600 => 600, activation_function),
    Dropout(0.8),
    Dense(600 => 600, activation_function),
    Dropout(0.8),
    Dense(600 => output_length, identity),
    ),
    Chain(
    Dense(input_length => 800, activation_function),
    Dropout(0.8),
    Dense(800 => 800, activation_function),
    Dropout(0.8),
    Dense(800 => 800, activation_function),
    Dropout(0.8),
    Dense(800 => output_length, identity),
    ),
    Chain(
    Dense(input_length => 1000, activation_function),
    Dropout(0.8),
    Dense(1000 => 1000, activation_function),
    Dropout(0.8),
    Dense(1000 => 1000, activation_function),
    Dropout(0.8),
    Dense(1000 => output_length, identity),
    ),
    Chain(
    Dense(input_length => 400, activation_function),
    Dropout(0.8),
    Dense(400 => 200, activation_function),
    Dropout(0.8),
    Dense(200 => 200, activation_function),
    Dropout(0.8),
    Dense(200 => output_length, identity),
    ),
    Chain(
    Dense(input_length => 600, activation_function),
    Dropout(0.8),
    Dense(600 => 400, activation_function),
    Dropout(0.8),
    Dense(400 => 400, activation_function),
    Dropout(0.8),
    Dense(400 => output_length, identity),
    ),
    Chain(
    Dense(input_length => 800, activation_function),
    Dropout(0.8),
    Dense(800 => 600, activation_function),
    Dropout(0.8),
    Dense(600 => 600, activation_function),
    Dropout(0.8),
    Dense(600 => output_length, identity),
    ),
    Chain(
    Dense(input_length => 1000, activation_function),
    Dropout(0.8),
    Dense(1000 => 800, activation_function),
    Dropout(0.8),
    Dense(800 => 800, activation_function),
    Dropout(0.8),
    Dense(800 => output_length, identity),
    ),
     ])
end

models_perTimeSlice = vcat(models_perTimeSlice...) |>f64;

## Describing parameter regions

In [ ]:
learning_rates = [1e-3];

In [ ]:
optimizers = []
push!(optimizers,[Flux.AdaGrad(), Flux.AdaDelta(), Flux.AMSGrad(), Flux.NAdam()])
for learning_rate in learning_rates
    push!(optimizers,
        [
            Flux.Adam(learning_rate),
            Flux.RAdam(learning_rate),
            Flux.AdaMax(learning_rate),
            Flux.AdamW(learning_rate),
            Flux.OAdam(learning_rate)
            ])
end
optimizers = vcat(optimizers...);

In [ ]:
function loss_mse(flux_model, x, y)
    batch_size = size(x)[2]
    ŷ = flux_model(x)
    
    return Flux.mse(ŷ, y, agg=sum)
end

function loss_mae(flux_model, x, y)
    batch_size = size(x)[2]
    ŷ = flux_model(x)
    
    return Flux.mae(ŷ, y, agg=sum)
end

In [ ]:
loss_functions = [loss_mse, loss_mae]

epochs = [150]

batch_sizes = [64];

In [ ]:
variables = ["Model", "Optimizer", "Loss function", "Epoch", "Batch size"];

In [ ]:
n_combinations = length(models_perTimeSlice) * length(loss_functions) * length(epochs) * length(batch_sizes) * length(optimizers)

## Creating Matrix with all combinations

In [ ]:
curr_date = now()

output_directory = "/Users/lukasgeyer/Studium/Computational Sciences/Masterarbeit/Tool Allesandro/repo/LmaPredict/gridSearch/eachTimeSlice/"

combination_matrix_name = output_directory * "models_$curr_date.csv"
results_matrix_name = output_directory * "results_$curr_date.csv"

In [ ]:
n_combinations = length(models_perTimeSlice) * length(loss_functions) * length(epochs) * length(batch_sizes) * length(optimizers)
combinations_matrix = Matrix{Any}(undef, n_combinations, length(variables))

index = 1
for model in models_perTimeSlice
    for optimizer in optimizers
        for loss_function in loss_functions
            for epoch in epochs
                for batch_size in batch_sizes
                    combinations_matrix[index,:] = [model, optimizer, loss_function, epoch, batch_size]
                    index +=1
                end
            end
        end
    end
end

CSV.write(combination_matrix_name, Tables.table(combinations_matrix), header=variables, delim=";")

# Perfoming Grid-Search

In [ ]:
scores_table_header = [
    "Index",
    "Min(max. R-Score)",
    "Avrg(max. R-Score)",
    "Std(max. R-Score)",
    "Avrg(epoch of max R-Score)",
    "Std(epoch of max R-Score)"
]
scores_mat = Matrix{Float64}(undef, n_combinations, length(scores_table_header))

for i in 1:n_combinations
    model_perTimeSlice = combinations_matrix[i,1] 
    optimizer = combinations_matrix[i,2]
    loss_function = combinations_matrix[i,3]
    epochs = combinations_matrix[i,4]
    batch_size = combinations_matrix[i,5]

    models = [model_perTimeSlice for i in 1:TVALS]

    max_R = Array{Float64}(undef, k_fold)
    epoch_of_max_R = Array{Float64}(undef, k_fold)
    for k in 1:k_fold
        mean_train = repeat(mean(re_data_train[k], dims=2), 1, test_size)
    
        Random.seed!(20)
    
        loaders = [Flux.DataLoader((input_data_train_standardized[k][i], output_data_train_standardized[k][i]),
                batchsize=batch_size,
                shuffle=true
                ) for i in 1:TVALS]
    
    
        R_scores = zeros(epochs)
        function training()
            losses = []
            for epoch in 1:epochs
                out_of_sample_predictions = Matrix{Float64}(undef, TVALS, test_size)
                
                for (i, model) in enumerate(models)
                    optim = Flux.setup(optimizer, model)
                    
                    for (x, y) in loaders[i]
                        grads = gradient(m -> loss_function(m, x, y), model)
                        Flux.update!(optim, model, grads[1])
                    end
                end
    
                for j in 1:TVALS
                    mean_output_train = mean(output_shape_train[k][j])
                    std_output_train = std(output_shape_train[k][j])
    
                    out_of_sample_predictions[j,:] = (models[j](input_data_test_standardized[k][j]) .* std_output_train) .+ mean_output_train
                end
                
                R_scores[epoch] = 1 - (Flux.mse(out_of_sample_predictions, output_shape_test, agg=sum) / Flux.mse(mean_train, output_shape_test, agg=sum))
    
            end
        end
    
        training()
    
        max_R[k] = maximum(R_scores)
        epoch_of_max_R[k] = argmax(R_scores)
    end
    
    scores_mat[i,:] = [
        i,
        minimum(max_R),
        mean(max_R),
        std(max_R),
        mean(epoch_of_max_R),
        std(epoch_of_max_R)
    ]
end

CSV.write(results_matrix_name, Tables.table(scores_mat), header=scores_table_header, delim=";")

In [ ]:
CSV.write(results_matrix_name, Tables.table(scores_mat), header=scores_table_header, delim=";")

# Evaluating results

In [ ]:
scores_mat = CSV.File(open(results_matrix_name); header=1, types=Float64) |> CSV.Tables.matrix

In [ ]:
function getParams(model::Chain)
    params = 0
    for layer in model
        params += sum(length, Flux.params(layer); init=0.0)
    end

    return params
end

In [ ]:
params = map(model -> getParams(model), combinations_matrix[:,1])
max_R = scores_mat[:,2]
epoch_R = scores_mat[:,3]

params_maxR_epochR_optimizer = hcat(hcat(hcat(params, max_R), epoch_R),combinations_matrix[:,2]);

In [ ]:
params_maxR_absSmallerTen = params_maxR_epochR_optimizer[(abs.(params_maxR_epochR_optimizer[:,2]) .< 10),:]
params_maxR_biggerZero = params_maxR_epochR_optimizer[(params_maxR_epochR_optimizer[:,2] .> 0),:]

scatter(params_maxR_biggerZero[:,1], params_maxR_biggerZero[:,2],
    size=(1000,800),
    xticks=([0,100_000,1_000_000,2_000_000],["0", "100_000", "1_000_000", "2_000_000"]),
    label=:false
)
xlabel!("Number of paramaters to train")
ylabel!("Maximum R Score")

#savefig("gridSearch.png")

In [ ]:
optimizer = optimizers[7]
params_maxR_biggerZero_optimizer = permutedims(stack([params_maxR_biggerZero[i,:] for i=1:size(params_maxR_biggerZero)[1] if params_maxR_biggerZero[i,4] == optimizer]))

scatter(params_maxR_biggerZero_optimizer[:,1], params_maxR_biggerZero_optimizer[:,2], size=(1000,800))
xlabel!("Number of paramaters to train")
ylabel!("Maximum R Score")

In [ ]:
params_epochR_later130 = params_maxR_epochR[(params_maxR_epochR[:,3] .> 130),:]

scatter(params_epochR_later130[:,1], params_epochR_later130[:,3], size=(1000,800))
xlabel!("Number of paramaters to train")
ylabel!("Epoch of maximum R Score")

# Test suite

In [ ]:
R_biggerZero = scores_mat[(scores_mat[:,2] .> 0),:]
max_R = R_biggerZero[(R_biggerZero[:,2] .≈ maximum(R_biggerZero[:,2])), :]

In [ ]:
#index = 2967
index = Int(max_R[1])

optimizer = combinations_matrix[index,2]
loss_function = combinations_matrix[index,3]
epochs = combinations_matrix[index, 4]
batch_size = combinations_matrix[index, 5]

epochMaxR = Int(scores_mat[index,3])
model = combinations_matrix[index,1] 

In [ ]:
model = Chain(
  Dense(94 => 400, tanh),               # 38_000 parameters
  Dropout(0.8),
  Dense(400 => 47),                     # 18_847 parameters
)

In [ ]:
Random.seed!(20)

loader = Flux.DataLoader(
    (input_data_train_standardized, output_data_train_standardized),
    batchsize=batch_size,
    shuffle=true
)

optim = Flux.setup(optimizer, model)

R_scores = zeros(epochs)
function training()
    for e in 1:epochs
        for (x,y) in loader
            grads = gradient(m -> loss_function(m, x, y), model)
            Flux.update!(optim, model, grads[1])
        end
        out_of_sample_predictions = (model(input_data_test_standardized) .* std_output_train) .+ mean_output_train
        R_scores[e] = 1 - (Flux.mse(out_of_sample_predictions, output_shape_test, agg=sum) / Flux.mse(mean_train, output_shape_test, agg=sum))
    end
end

losses = training()

out_of_sample_predictions = (model(input_data_test_standardized) .* std_output_train) .+ mean_output_train
R = 1 - (Flux.mse(out_of_sample_predictions, output_shape_test, agg=sum) / Flux.mse(mean_train, output_shape_test, agg=sum))

In [ ]:
plot(R_scores)

In [ ]:
l = @layout [a b c; d e f; g h i]

c1 = rand([i for i in 1:500])
p1 = scatter(output_shape_test[:,c1], label="Actual")
scatter!(p1, out_of_sample_predictions[:,c1], label="Prediction", legend=:top)

c2 = rand([i for i in 1:500])
p2 = scatter(output_shape_test[:,c2], label="Actual")
scatter!(p2, out_of_sample_predictions[:,c2], label="Prediction", legend=:top)

c3 = rand([i for i in 1:500])
p3 = scatter(output_shape_test[:,c3], label="Actual")
scatter!(p3, out_of_sample_predictions[:,c3], label="Prediction", legend=:top, )

c4 = rand([i for i in 1:500])
p4 = scatter(output_shape_test[:,c4], label="Actual")
scatter!(p4, out_of_sample_predictions[:,c4], label="Prediction", legend=:top)

c5 = rand([i for i in 1:500])
p5 = scatter(output_shape_test[:,c5], label="Actual")
scatter!(p5, out_of_sample_predictions[:,c5], label="Prediction", legend=:top)

c6 = rand([i for i in 1:500])
p6 = scatter(output_shape_test[:,c6], label="Actual")
scatter!(p6, out_of_sample_predictions[:,c6], label="Prediction", legend=:top)

c7 = rand([i for i in 1:500])
p7 = scatter(output_shape_test[:,c7], label="Actual")
scatter!(p7, out_of_sample_predictions[:,c7], label="Prediction", legend=:top)

c8 = rand([i for i in 1:500])
p8 = scatter(output_shape_test[:,c8], label="Actual")
scatter!(p8, out_of_sample_predictions[:,c8], label="Prediction", legend=:top)

c9 = rand([i for i in 1:500])
p9 = scatter(output_shape_test[:,c9], label="Actual")
scatter!(p9, out_of_sample_predictions[:,c9], label="Prediction", legend=:top)

plot(p1, p2, p3, p4, p5, p6, p7, p8, p9, layout = l, size=(1200,1000), dpi=1000, markerstrokewidth = 0)
#savefig(joinpath(path_plot, "neural_network_test.pdf"))

In [ ]:
mean_target = mean.([output_shape_test[i,:] for i in 1:TVALS])
σ_target = std.([output_shape_test[i,:] for i in 1:TVALS]) ./ sqrt(test_size - 1)
        
mean_predicted = mean.([out_of_sample_predictions[i,:] for i in 1:TVALS])
σ_predicted = std.([out_of_sample_predictions[i,:] for i in 1:TVALS]) ./ sqrt(test_size - 1)

max_Δμ = maximum(abs.((mean_target - mean_predicted) ./ mean_target))

p = scatter(
        size=(1400,1000),
        dpi = 1000,
        thickness_scaling = 1.5,
)
    
scatter!(p,
    mean_target[7:41],
    yerr=σ_target[7:41],
    label="actual",
    legend=:bottom,
    linecolor=:blue,
    marker=:xcross,
    markersize = 5,
    markerstrokewidth = 0.3
)
scatter!(p,
    mean_predicted[7:41],
    yerr=σ_predicted[7:41],
    label="predicted",
    legend=:bottom,
    linecolor=:red,
    marker =:+,
    markersize = 5,
    markerstrokewidth = 0.3
)

# Bias-Correction

### Applying bias correction on a percentage of the test data

In [ ]:
num_tests = 100
percentages = [0.0, 0.01, 0.02, 0.04, 0.08, 0.1, 0.2]
seeds = rand([i for i in 1:1_000_000], num_tests)

for percentage in percentages
    num_no_overlappings = zeros(num_tests)
    χ²_s = zeros(num_tests)
    χ²_train_s = zeros(num_tests)
    i = 1
    for test in 1:num_tests
        n_configs = Int(test_size * percentage)

        Random.seed!(seeds[test])
        configs = sort!(sample([i for i in 1:test_size], n_configs, replace = false))
        
        uncorr_target_configs = stack(deleteat!([output_shape_test[:,i] for i in 1:test_size],configs), dims=2)
        
        mean_target = mean(uncorr_target_configs, dims=2)
        σ_mean_target = std(uncorr_target_configs, dims=2) ./ sqrt(test_size - 1 - n_configs)

        mean_target_train = mean(output_shape_train, dims=2)
        σ_mean_target_train = std(output_shape_train, dims=2) ./ sqrt(train_size - 1)
            
        mean_predicted = mean(out_of_sample_predictions, dims=2)
        σ_predicted = std(out_of_sample_predictions, dims=2) ./ sqrt(test_size - 1)
        
        if percentage > 0
            bias_correction = mean(hcat([[out_of_sample_predictions[:,i] - output_shape_test[:,i] for i in configs][i] for i in 1:length(configs)]...), dims=2)
            σ_bc = std(hcat([[out_of_sample_predictions[:,i] - output_shape_test[:,i] for i in configs][i] for i in 1:length(configs)]...), dims=2) ./ sqrt(n_configs - 1)
            
            mean_target_train = mean(hcat(output_shape_train, hcat([output_shape_test[:,i] for i in configs]...)), dims=2)
            σ_mean_target_train = std(hcat(output_shape_train, hcat([output_shape_test[:,i] for i in configs]...)), dims=2) ./ sqrt(train_size + n_configs - 1)
        else
            bias_correction = zeros(output_length)
            σ_bc = zeros(output_length)
        end
        
        mean_predicted = mean_predicted - bias_correction
        σ_pred_bc = σ_predicted + σ_bc
        
        χ²_s[i] = sum(((mean_predicted - mean_target) ./ sqrt.(σ_pred_bc.^2 + σ_mean_target.^2)).^2)
        χ²_train_s[i] = sum(((mean_target_train - mean_target) ./ sqrt.(σ_mean_target_train.^2 + σ_mean_target.^2)).^2)

        no_overlaps = 0
        for t in 1:output_length
            predicted_min = mean_predicted[t] - σ_pred_bc[t]
            predicted_max = mean_predicted[t] + σ_pred_bc[t]
        
            target_min = mean_target[t] - σ_mean_target[t]
            target_max = mean_target[t] + σ_mean_target[t]
        
            if predicted_min > target_max || predicted_max < target_min
                no_overlaps += 1
            end
        end

        num_no_overlappings[i] = no_overlaps
        i += 1
    end

    mean_no_overlappings = mean(num_no_overlappings)
    mean_χ² = mean(χ²_s)
    maximum_χ² = maximum(χ²_s)
    mean_χ²_train = mean(χ²_train_s)
    maximum_χ²_train = maximum(χ²_train_s)
    println("Percentage used for bias-correction: $percentage")
    println("Average number of non-overlappings: $mean_no_overlappings")
    println("<χ²>: $mean_χ², max χ²: $maximum_χ²")
    println("<χ²_train>: $mean_χ²_train, max χ²_train: $maximum_χ²_train\n")
end     

In [ ]:
x_start = 8
x_end = 40

percentage = 0.1
n_configs = Int(test_size * percentage)        
configs = sort!(sample([i for i in 1:test_size], n_configs, replace = false))
        
uncorr_target_configs = stack(deleteat!([output_shape_test[:,i] for i in 1:test_size],configs), dims=2)
        
mean_target = mean(uncorr_target_configs, dims=2)
σ_mean_target = std(uncorr_target_configs, dims=2) ./ sqrt(test_size - 1 - n_configs)

mean_target_train = mean(output_shape_train, dims=2)
σ_mean_target_train = std(output_shape_train, dims=2) ./ sqrt(train_size - 1)
            
mean_predicted = mean(out_of_sample_predictions, dims=2)
σ_predicted = std(out_of_sample_predictions, dims=2) ./ sqrt(test_size - 1)
        
if percentage > 0
    bias_correction = mean(hcat([[out_of_sample_predictions[:,i] - output_shape_test[:,i] for i in configs][i] for i in 1:length(configs)]...), dims=2)
    σ_bc = std(hcat([[out_of_sample_predictions[:,i] - output_shape_test[:,i] for i in configs][i] for i in 1:length(configs)]...), dims=2) ./ sqrt(n_configs - 1)

    mean_target_train = mean(hcat(output_shape_train,hcat([output_shape_test[:,i] for i in configs]...)), dims=2)
    σ_mean_target_train = std(hcat(output_shape_train,hcat([output_shape_test[:,i] for i in configs]...)), dims=2) ./ sqrt(train_size + n_configs - 1)
else
    bias_correction = zeros(output_length)
    σ_bc = zeros(output_length)
end
        
mean_predicted = mean_predicted - bias_correction
σ_pred_bc = σ_predicted + σ_bc

no_overlaps = 0
for t in 1:output_length
    predicted_min = mean_predicted[t] - σ_pred_bc[t]
    predicted_max = mean_predicted[t] + σ_pred_bc[t]
        
    target_min = mean_target[t] - σ_mean_target[t]
    target_max = mean_target[t] + σ_mean_target[t]
        
    if predicted_min > target_max || predicted_max < target_min
        println("Errors are not overlapping on t = $t")
    end
end

χ² = sum(((mean_predicted - mean_target) ./ sqrt.(σ_pred_bc.^2 + σ_mean_target.^2)).^2)
χ²_train = sum(((mean_target_train - mean_target) ./ sqrt.(σ_mean_target_train.^2 + σ_mean_target.^2)).^2)
println("χ² = ", χ²)
println("χ²_train = ", χ²_train)

p = scatter(
    size=(1400,1000),
    dpi = 1000,
    thickness_scaling = 1.6,
    title="bc: $n_configs"
)
    
#scatter!(p,
#    mean_target[x_start:x_end],
#    yerr=σ_mean_target[x_start:x_end],
#    label="actual",
#    legend=:bottom,
#    linecolor=:blue,
#    marker=:xcross,
#    markersize = 2,
#    markerstrokewidth = 0.3,
#    xticks=([i for i in 1:2:(x_end-x_start)+1],["$i" for i in x_start:2:x_end])
#)

scatter!(p,
    mean_predicted[x_start:x_end],
    yerr=σ_pred_bc[x_start:x_end],
    label="predicted",
    legend=:bottom,
    linecolor=:red,
    marker =:+,
    markersize = 2,
    markerstrokewidth = 0.3,
    xticks=([i for i in 1:2:(x_end-x_start)+1],["$i" for i in x_start:2:x_end])
)

scatter!(p,
    mean_target_train[x_start:x_end],
    yerr=σ_mean_target_train[x_start:x_end],
    label="actual (training set)",
    legend=:bottom,
    linecolor=:yellow,
    marker=:xcross,
    markersize = 2,
    markerstrokewidth = 0.3,
    xticks=([i for i in 1:2:(x_end-x_start)+1],["$i" for i in x_start:2:x_end])
)

xlabel!(p,L"t/a")
ylabel!(p,L"C^{re}(x_0,y_0)")
#savefig(p,"prediction_mean_bc.png")